In [1]:
import pandas as pd
import numpy as np

In [2]:
orders = pd.read_csv("data/orders.csv")
order_products = pd.read_csv("data/order_products.csv")
joint = orders.merge(order_products, how="left", on="order_id")
joint.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order
0,2539329,1,1,2,8,NaN,196.0,1.0
1,2539329,1,1,2,8,NaN,14084.0,2.0
2,2539329,1,1,2,8,NaN,12427.0,3.0
3,2539329,1,1,2,8,NaN,26088.0,4.0
4,2539329,1,1,2,8,NaN,26405.0,5.0


<b>--- Create a new column "eval_set" and fill with NaN <br/>
--- If product_id is null, set eval_set="test".</b>

In [3]:
joint["eval_set"] = np.nan
# set "test" where product_id is null
joint["eval_set"][joint["product_id"].isnull()] = "test"


<ipython-input-3-7fc09d16c575>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joint["eval_set"][joint["product_id"].isnull()] = "test"


<b>--- Check the last order of each user.<br/>
--- When eval_set="test", product_id=NaN, add_to_cart_order=Nan.<br/>
--- Seems fine.</b>

In [4]:
joint.loc[joint.reset_index().groupby(["user_id"])["order_number"].idxmax()]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
59,1187899,1,11,4,8,14.0,196.0,1.0,NaN
265,1492625,2,15,1,11,30.0,22963.0,1.0,NaN
384,2774568,3,13,5,15,11.0,NaN,NaN,test
403,329954,4,6,3,12,30.0,NaN,NaN,test
441,2196797,5,5,0,11,6.0,15349.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...
33892762,1716008,206205,4,1,16,10.0,27845.0,1.0,NaN
33893066,1043943,206206,68,0,20,0.0,NaN,NaN,test
33893290,2821651,206207,17,2,13,14.0,NaN,NaN,test
33893968,803273,206208,50,5,11,4.0,NaN,NaN,test


<b>--- In the above dataframe, rows where eval_set = NaN should be assigned eval_set="train".<br/>
--- That is because the last orders of a user with a product_id is train data.<br/>
--- Exact those row indexes and assign.</b>

In [5]:
last_orders = joint.loc[joint.reset_index().groupby(["user_id"])["order_number"].idxmax()]["order_id"]

# row index of last orders
last_orders.index.tolist()
len(last_orders.index.tolist())

206209

In [6]:
# set "train"
joint.iloc[last_orders.index.tolist(),-1] = joint.iloc[last_orders.index.tolist(),-1].fillna("train")

<b>--- Check again the last order of each user.<br/>
--- Now we have "test" and "train".<br/>
--- Seems fine.</b>

In [7]:
joint.loc[joint.reset_index().groupby(["user_id"])["order_number"].idxmax()]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
59,1187899,1,11,4,8,14.0,196.0,1.0,train
265,1492625,2,15,1,11,30.0,22963.0,1.0,train
384,2774568,3,13,5,15,11.0,NaN,NaN,test
403,329954,4,6,3,12,30.0,NaN,NaN,test
441,2196797,5,5,0,11,6.0,15349.0,1.0,train
...,...,...,...,...,...,...,...,...,...
33892762,1716008,206205,4,1,16,10.0,27845.0,1.0,train
33893066,1043943,206206,68,0,20,0.0,NaN,NaN,test
33893290,2821651,206207,17,2,13,14.0,NaN,NaN,test
33893968,803273,206208,50,5,11,4.0,NaN,NaN,test


<b>--- Set all other remaining rows as eval_set="prior".<br/></b>

In [8]:
values = {'eval_set': "prior"}
joint.fillna(value=values, inplace=True)

<b>--- Final check.<br/></b>

In [11]:
joint['eval_set'].isnull().sum()

0

In [15]:
test = joint[joint["eval_set"] == "test"]
test

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
384,2774568,3,13,5,15,11.0,NaN,NaN,test
403,329954,4,6,3,12,30.0,NaN,NaN,test
464,1528013,6,4,3,16,22.0,NaN,NaN,test
1086,1376945,11,8,6,11,8.0,NaN,NaN,test
1161,1356845,12,6,1,20,30.0,NaN,NaN,test
...,...,...,...,...,...,...,...,...,...
33892542,2728930,206202,23,2,17,6.0,NaN,NaN,test
33892729,350108,206204,5,4,14,14.0,NaN,NaN,test
33893066,1043943,206206,68,0,20,0.0,NaN,NaN,test
33893290,2821651,206207,17,2,13,14.0,NaN,NaN,test


In [17]:
train = joint[joint["eval_set"] == "train"]
train

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
59,1187899,1,11,4,8,14.0,196.0,1.0,train
265,1492625,2,15,1,11,30.0,22963.0,1.0,train
441,2196797,5,5,0,11,6.0,15349.0,1.0,train
671,525192,7,21,2,11,6.0,12053.0,1.0,train
729,880375,8,4,1,14,10.0,15937.0,1.0,train
...,...,...,...,...,...,...,...,...,...
33891619,2585586,206199,20,2,16,30.0,29429.0,1.0,train
33891920,943915,206200,24,6,19,6.0,22828.0,1.0,train
33892662,2371631,206203,6,4,19,30.0,15693.0,1.0,train
33892762,1716008,206205,4,1,16,10.0,27845.0,1.0,train


In [18]:
prior = joint[joint["eval_set"] == "prior"]
prior

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,eval_set
0,2539329,1,1,2,8,NaN,196.0,1.0,prior
1,2539329,1,1,2,8,NaN,14084.0,2.0,prior
2,2539329,1,1,2,8,NaN,12427.0,3.0,prior
3,2539329,1,1,2,8,NaN,26088.0,4.0,prior
4,2539329,1,1,2,8,NaN,26405.0,5.0,prior
...,...,...,...,...,...,...,...,...,...
33894101,272231,206209,14,6,14,30.0,40603.0,4.0,prior
33894102,272231,206209,14,6,14,30.0,15655.0,5.0,prior
33894103,272231,206209,14,6,14,30.0,42606.0,6.0,prior
33894104,272231,206209,14,6,14,30.0,37966.0,7.0,prior


In [20]:
print("rows of prior: ", prior.shape[0])
print("rows of train: ", train.shape[0])
print("rows of test: ", test.shape[0])

rows of prior:  33687897
rows of train:  131209
rows of test:  75000


In [21]:
joint.shape

(33894106, 9)

In [22]:
orders.shape

(3421083, 6)

In [24]:
order_products.shape

(33819106, 3)